In [3]:
from datetime import datetime, date
import numpy as np
import pandas as pd
from scipy.stats import norm
import QuantLib as ql
from pysabr import Hagan2002LognormalSABR, Hagan2002NormalSABR

In [4]:
def cal_trading_days(start_date, end_date):
    """
    计算两个日期之间的交易日数量
    """
    trading_calendar = pd.read_pickle('data/trading_calendar.pkl')
    if isinstance(start_date, str):
        start_date = start_date
    elif isinstance(start_date, (datetime, date, pd.Timestamp)):
        start_date = pd.to_datetime(start_date).strftime('%Y%m%d')

    if isinstance(end_date, str):
        end_date = end_date
    elif isinstance(end_date, (datetime, date, pd.Timestamp)):
        end_date = pd.to_datetime(end_date).strftime('%Y%m%d')
    trading_days = trading_calendar[(trading_calendar[0] >= start_date) & (trading_calendar[0] <= end_date)]
    return len(trading_days)

In [5]:
def black_scholes_price(S, K, sigma, T, r, q, is_call=True):
    """
    Black-Scholes期权定价公式
    S: 标的现价
    K: 行权价
    sigma: 隐含波动率（对数正态）
    T: 期限（年）
    r: 无风险利率
    q: 标的资产分红率（股票适用）
    is_call: 看涨/看跌期权
    """
    F = S * np.exp((r - q) * T)  # 远期价格
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if is_call:
        price = np.exp(-r * T) * (F * norm.cdf(d1) - K * norm.cdf(d2))
    else:
        price = np.exp(-r * T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    return price

In [24]:
S = 4792     # 标的现货价格
K = 7000     # 行权价
T = cal_trading_days('20250624', '20250917') / 252  # 年化到期时间
r = 0.02     # 无风险利率
sigma_atm = 0.20 * S   # ATM波动率
beta = 0.5   # 弹性参数
rho = -0.5   # 价格-波动率相关性
volvol = 0.8  # 波动率的波动率

# 计算远期价格（假设无分红，远期=现货）
f = S #* np.exp(r * T)

# 初始化SABR模型（这里使用对数正态SABR）
sabr_ln = Hagan2002LognormalSABR(
    f=f ,          # 远期价格
    shift=0,      # 位移（商品期权通常为0）
    t=T,          # 到期时间
    v_atm_n=sigma_atm ,  # ATM波动率
    beta=beta,    # 弹性参数
    rho=rho,      # 相关系数
    volvol=volvol # 波动率的波动率
)

sabr_n = Hagan2002NormalSABR(
    f=f ,          # 远期价格
    shift=0,      # 位移（商品期权通常为0）
    t=T,          # 到期时间
    v_atm_n=sigma_atm ,  # ATM波动率
    beta=beta,    # 弹性参数
    rho=rho,      # 相关系数
    volvol=volvol # 波动率的波动率
)

# 计算给定行权价的隐含波动率
ln_vol = sabr_ln.lognormal_vol(K) # 获取正态波动率
n_vol = sabr_n.lognormal_vol(K) # 获取对数正态波动率
# 或者使用对数正态波动率：sabr_ln.lognormal_vol(K)

print(f"标的现货价格: {S}")
print(f"行权价: {K}")
print(f"到期时间(年): {T:.4f}")
print(f"无风险利率: {r:.2%}")
print(f"计算得到的ln隐含波动率: {ln_vol:.2%}")
print(f"计算得到的n隐含波动率: {n_vol:.2%}")

标的现货价格: 4792
行权价: 7000
到期时间(年): 0.2460
无风险利率: 2.00%
计算得到的ln隐含波动率: 18.32%
计算得到的n隐含波动率: 18.36%


对数正态SABR隐含波动率（Black-Scholes）: 17.52%


In [25]:
strikes = np.array([4250, 4300, 4350, 4400, 4450, 4500, 4550, 4600, 4650, 4700, 4750, 4800, 4850, 4900, 4950, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800])
sigmas = np.array([0.5912, 0.2404,0.2261,0.1926, 0.1982, 0.1966,0.1891, 0.1866, 0.1854, 0.1858, 0.1835, 0.1832, 0.1855, 0.1858, 0.1888,0.1908, 0.1996, 0.2094, 0.2205, 0.2318,0.2452, 0.2586,0.2740, 0.2818])

In [26]:
[alpha, rho, volvol] = Hagan2002LognormalSABR(f=4766, shift=0, t=0.22 , beta=0.5).fit(strikes, sigmas)

In [27]:
[alpha, rho, volvol]

[np.float64(0.12009724417035736),
 np.float64(-0.35686255769273717),
 np.float64(0.04054373912676577)]

In [32]:
# 参数设置
S = 4766           # 标的现货价格
K = 5800           # 行权价
T = 0.22           # 年化到期时间
sigma_ln_atm = 0.186 * S # ATM对数正态波动率
beta = 0.12009724417035736        # 弹性参数
rho = -0.35686255769273717        # 价格-波动率相关性
volvol = 0.04054373912676577     # 波动率的波动率

# 初始化对数正态SABR模型

sabr_ln = Hagan2002LognormalSABR(
    f=S,           # 远期价格（假设无分红，F=S）
    shift=0,       # 位移（通常为0）
    t=T,           # 到期时间
    v_atm_n=sigma_ln_atm ,  # 初始波动率（对数正态）
    beta=beta,     # 弹性参数
    rho=rho,       # 相关系数
    volvol=volvol  # 波动率的波动率
)

# 计算行权价K处的对数正态隐含波动率
ln_vol = sabr_ln.lognormal_vol(K)
print(f"对数正态SABR隐含波动率（Black-Scholes）: {ln_vol:.2%}")

对数正态SABR隐含波动率（Black-Scholes）: 16.91%
